In [2]:
import os
from google.cloud import bigquery as bq
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'itr-aimasteryhub-lab-1a116262496d.json'
client = bq.Client(project="itr-aimasteryhub-lab")
 
datasets = list(client.list_datasets())  # Make an API request.
 
for dataset in datasets:
    print(dataset.dataset_id)

conference_data
news_agent


In [ ]:
from scrapers.parsers.aicon_infoq import AiconInfoqScraper
from bigquery.client import BigQueryClient

scraper = AiconInfoqScraper(
                use_bigquery=True,
                bq_credentials='itr-aimasteryhub-lab-1a116262496d.json',
                bq_project_id="itr-aimasteryhub-lab"
            )

scraper.run()

已創建數據集 conference_data
已創建表 conference_data.sessions

開始爬取 202503 AICon Shanghai
開始爬取網頁: https://aicon.infoq.cn/2025/shanghai/track
找到 56 個演講題目
正在處理第 1/56 個項目: 端侧智能的模型架构设计与算法改进
找到PDF連結: http://ppt.geekbang.org/slide/download?cid=158&pid=4989
正在處理第 2/56 個項目: 面向端侧大模型的原生 AI SoC 设计与实践
未找到PDF連結或查找過程出錯: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[contains(@class, "item ppt") or contains(@class, "icon-ppt")]"}
  (Session info: chrome=137.0.7151.69); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000109cf07a8 chromedriver + 5892008
1   chromedriver                        0x0000000109ce7efa chromedriver + 5857018
2   chromedriver                        0x00000001097b8400 chromedriver + 418816
3   chromedriver                        0x000000010980a22b chromedriver + 754219
4   chromedriver            

'202503 AICon Shanghai_20250611.xlsx'

In [8]:
# Read from conference_data.sessions table in BigQuery
try:
    # First check if we have the db-dtypes package
    import importlib.util
    spec = importlib.util.find_spec('db_dtypes')
    if spec is None:
        print("The db-dtypes package is not available. Please run the cell below to install it and restart the kernel.")
    else:
        query = """SELECT * FROM `itr-aimasteryhub-lab.conference_data.sessions`"""
        conference_data_sessions = client.query(query).to_dataframe()
        
        # Show original data
        print(f"Original data has {len(conference_data_sessions)} rows")
        
        # Delete row 0 (the first row)
        if len(conference_data_sessions) > 0:  # Make sure we have at least one row
            filtered_sessions = conference_data_sessions.drop([0]).reset_index(drop=True)
            print(f"After deletion, data has {len(filtered_sessions)} rows")
            
            # Display the filtered data
            filtered_sessions
        else:
            print("No rows to delete")
            conference_data_sessions
except Exception as e:
    print(f"Error: {str(e)}")
    print("\nIf the error is about missing db-dtypes, please run the cell below to install it and restart the kernel.")
    import sys
    print(f"Python version: {sys.version}")
    print(f"Path: {sys.path}")
    # Show more diagnostic information
    import subprocess
    print("\nInstalled packages:")
    !pip list | grep -E "db-dtypes|pandas|pyarrow"

Error: Please install the 'db-dtypes' package to use this function.

If the error is about missing db-dtypes, please run the cell below to install it and restart the kernel.
Python version: 3.13.1 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 10:35:08) [Clang 14.0.6 ]
Path: ['/Users/cfh00896102/miniconda3/envs/crawl4ai/lib/python313.zip', '/Users/cfh00896102/miniconda3/envs/crawl4ai/lib/python3.13', '/Users/cfh00896102/miniconda3/envs/crawl4ai/lib/python3.13/lib-dynload', '', '/Users/cfh00896102/miniconda3/envs/crawl4ai/lib/python3.13/site-packages']

Installed packages:
db-dtypes                    1.4.3
pandas                       2.2.3
pyarrow                      20.0.0
db-dtypes                    1.4.3
pandas                       2.2.3
pyarrow                      20.0.0


In [4]:
# Install the required db-dtypes package for BigQuery to pandas conversion
!pip install db-dtypes

# IMPORTANT: After running this cell, please restart the kernel
# You can do this by clicking on the 'Kernel' menu and selecting 'Restart'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/32.3 MB ? eta -:--:--Downloading pyarrow-20.0.0-cp313-cp313-macosx_12_0_x86_64.whl (32.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.3/32.3 MB 4.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.3/32.3 MB 4.2 MB/s eta 0:00:00


In [6]:
# Alternative approach: Delete the first row directly with SQL
# This avoids the need for pandas and db-dtypes
query = """
WITH indexed_data AS (
  SELECT *, ROW_NUMBER() OVER() AS row_num
  FROM `itr-aimasteryhub-lab.conference_data.sessions`
)
SELECT * EXCEPT(row_num)
FROM indexed_data
WHERE row_num != 1  -- Skip the first row (row_num = 1)
"""

try:
    # Run the query and display results
    result = client.query(query)
    # Convert to list to view the data
    rows = list(result)
    print(f"Number of rows after excluding the first row: {len(rows)}")
    
    # Display column names
    if rows:
        print("\nColumns:")
        for field in result.schema:
            print(f"- {field.name} ({field.field_type})")
    
    # Display a few rows
    print("\nFirst few rows:")
    for i, row in enumerate(rows[:5]):
        print(f"\nRow {i+1}:")
        for key, value in row.items():
            print(f"  {key}: {value}")
except Exception as e:
    print(f"Error: {str(e)}")

Number of rows after excluding the first row: 58

Columns:
Error: 'NoneType' object is not iterable
